# 필요 함수 import

In [1]:
import requests
from bs4 import BeautifulSoup

import re
from collections import Counter
import nltk
from konlpy.tag import Okt

import pandas as pd
import numpy as np

import pickle

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gyuu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6중식요리 502490
- 7기타요리 502491

In [2]:
# 쿠팡 페이지 호출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [3]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#해당 카테고리의 페이지 전체 수 가져오기
def page_number(categoryId, user_agt):
    URL = 'https://www.coupang.com/np/categories/'+categoryId+"?page=1"
    cg_soup_base = get_coupang_item(URL, user_agt)
    page_all = cg_soup_base.select('#product-list-paging > div > a')
    page_list = []
    for page_num in page_all:
        page_num = page_num.text
        try:
            page_num = int(page_num)
            page_list.append(page_num)
        except:
            pass
    return page_list[-1]


#1차 쿠팡 상품 리스트 전처리 함수
def p_list_preprocessing(cg_soup, category_num):
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item

## 쿠팡 상품 리스트 추출 실행부
> 밀키트 추가 필터링 후 전체 크롤링

- 추가 전처리 해야하는 부분
1. 100g당 가격 결측치 처리
2. 구성정보 텍스트 전처리 : 명사 추출 후 재료만 남기기
3. 구성정보 결측치 채우기 : 텍스트로 구성정보가 존재하는 경우만 >> 그외 결측치 삭제
4. 상품명에서 음식명만 추출 > 세부 카테고리 정리

In [4]:
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502490':6, '502491':7}
category_keys = list(category.keys())

#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)

#쿠팡 상품 리스트 추출
cg_url = 'https://www.coupang.com/np/categories/'
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

for categoryId in category_keys:
    page_num = page_number(categoryId, user_agt) #해당 카테고리의 페이지 마지막 페이지 번호 추출
    for page in range(1, page_num+1):
        URL = cg_url+categoryId+"?page="+str(page)+'&filter=1%23attr_12406%2419087%40DEFAULT'
        
        #페이지 호출
        cg_soup_base = get_coupang_item(URL, user_agt)
        cg_soup2 = cg_soup_base.select("#productList li")  

        #상품 리스트 추출 및 전처리
        for cg_soup in cg_soup2:
            coupang_item = p_list_preprocessing(cg_soup, category_number(categoryId, category))
            coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                372 non-null    object
 1   상품id                 372 non-null    object
 2   data-item-id         372 non-null    object
 3   data-vendor-item-id  372 non-null    object
 4   상품                   372 non-null    object
 5   상품명                  372 non-null    object
 6   정가                   372 non-null    object
 7   할인율                  372 non-null    object
 8   판매가                  372 non-null    object
 9   100g당_가격             329 non-null    object
 10  별점                   372 non-null    object
 11  리뷰수                  372 non-null    object
 12  품절여부                 372 non-null    object
dtypes: object(13)
memory usage: 37.9+ KB


In [5]:
#전체 중복행 확인
coupang_items.loc[coupang_items.duplicated(subset=['상품id']), :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부


In [6]:
#중복행 삭제
coupang_items.drop_duplicates('상품id', inplace=True)
coupang_items.reset_index(drop=True, inplace=True)

# 2차 : 구성 정보 추출
- 상품id, 구성정보

In [7]:
#2차 : 구성정보 추출 함수
def p_c_extraction(productId, p_c_page):
    product_composition = '0'
    for item in p_c_page:
        s = item.text.replace('\n','').strip()
        if product_composition == '2': #3: 재료부분만 추출
            product_composition = s
        elif product_composition =='1':#2: 불필요한 text 제외
            product_composition='2'
        elif '구성 정보' in s :         #1: '구성 정보' 존재여부 확인
            product_composition = '1'

    #4: 없으면 결측값 처리
    if product_composition.isdigit():
        product_composition = np.NaN
    
    data_item = pd.DataFrame({'상품id':productId, '구성정보':product_composition}, index=[0])
    return data_item

## 2차 실행부

In [8]:
#2차 실행부
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    #각 상품 상세페이지 호출
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    pdt_soup = get_coupang_item(URL, user_agt)
    
    #'구성정보' 추출
    p_c_page = pdt_soup.find('body').children
    data_item = p_c_extraction(productId, p_c_page)
    data_items = pd.concat([data_items, data_item], ignore_index=True)

data_items.head()

,상품id,구성정보
0,4926044090,NaN
1,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
2,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채..."
3,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다."
4,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성..."


# 상세페이지 이미지에서 구성정보 추출


In [9]:
def get_coupang_image(img_URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    page = requests.get(img_URL, headers= headers).text
    #coupang_image_soup = BeautifulSoup(page.text, "html.parser")
    #return coupang_image_soup
    return page

In [10]:
# 구성정보 페이지에서 전체 이미지 링크 추출 -> 리스트화
image_link = []
product_url = 'https://www.coupang.com/vp/products/'
image_productId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['상품id'].values)
image_dataitemId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['data-item-id'].values)
image_datavendorId = list(coupang_items.loc[coupang_items['상품명'].str.contains('곰곰|딜리조이|프렙')]['data-vendor-item-id'].values)

for product in range(len(image_productId)):
    img_URL = product_url + str(image_productId[product]) + '/items/' +str(image_dataitemId[product])+'/vendoritems/'+str(image_datavendorId[product])
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    page = get_coupang_image(img_URL, user_agt)

    link_start = []
    for text in re.finditer('//thumbnail',page):
        link_start.append(text.start())
        
    link_end = []
    for text in re.finditer('g"}],',page):
        link_end.append(text.start())
    
    for num in range(len(link_start)):
        image_link.append('https:' + page[int(link_start[num]):int(link_end[num]) + 1])
        

In [11]:
## 상세페이지에서 전체 이미지 저장
import urllib.request

count = 0
for url in image_link:
    count += 1
    path = "C:/coding/image/" + (str(count) + ".jpg")
    urllib.request.urlretrieve(url, path)


In [12]:
# 이미지에서 텍스트 추출 코드 with pytesserect
import pytesseract
import cv2 
import matplotlib.pyplot as plt
from PIL import Image

extracted_text = []
for num in range(1, count + 1):
    file = 'C:/coding/image/' + str(num) + '.jpg'
    image = Image.open(file)
    if image.size[1] >= 4000:
        croppedImage = image.crop((0,2000,image.size[0],3350))
        croppedImage.save('C:/coding/image/cropped' + str(num) + '.jpg')
        file = 'C:/coding/image/cropped' + str(num) + '.jpg'
        image = cv2.imread(file)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(rgb_image, lang='kor')
        extracted_text.append(text)
    else:
        image = cv2.imread(file)
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        text = pytesseract.image_to_string(rgb_image, lang='kor')
        extracted_text.append(text)

In [13]:
# 결측값 중 곰곰 밀키트 구성정보만 테서렉트로 이미지 텍스트 추출 후,
# 오타 교정 위해 전처리 작업
gomgom_text = [i for i in extracted_text if "구성되어 있어요"  in i] 
gomgom = []
for i in gomgom_text:
    sentence = re.sub('(<([^>]+)>)', '', i)
    sentence = re.sub('[^ ㄱ-ㅎㅏ-ㅣ각-힣]', ' ', sentence)
    gomgom.append(sentence)
    
gom_typo_list = [] 

for gom in gomgom:
    gg = gom.split(' ')
    gom_typo = []
    for i in range(len(gg)):
        if len(gg[i]) > 1:
            gom_typo.append(gg[i])
        else:
            pass
    gom_typo_list.append(gom_typo)

In [14]:
# gom_type_list 피클화
import pickle

with open('gom_typo.pkl','wb') as f:
    pickle.dump(gom_typo_list,f)

# 곰곰 구성재료 오타 교정 (Colab)

In [15]:
# !pip install symspellpy jamo hangul_utils
# !wget https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt
# from symspellpy import SymSpell, Verbosity
# from symspellpy_ko import KoSymSpell, Verbosity
# from hangul_utils import split_syllable_char, split_syllables, join_jamos
# from jamo import h2j, j2hcj
# import pandas as pd
# import unicodedata

# def to_jamos(text):
#   return j2hcj(h2j(text))


# 만개의 레시피 재료 목록 자소 분리
# vocab = pd.read_csv('만개의레시피_재료_목록_오타처리용.txt', sep = " ", names = ["term", "count"])
# vocab.head()
# vocab.info()
# vocab = vocab.dropna(axis=0)

# vocab.term = vocab.term.map(to_jamos)
# vocab.to_csv("만개의레시피_재료_목록_오타처리용_decomposed.txt", sep=" ", header=None, index=None)
# vocab.head()

In [16]:
# # 저장했던 리스트 불러옴
# with open('gom_typo.pkl','rb') as f:
#   gom = pickle.load(f)
  
  
# for g in gom:
#     idx = g.index('있어요')
#     del g[0:idx + 1]

# for g in gom:
#   if '일은' in g :
#     idx = g.index('일은')
#     del g[idx :]

#   elif 'ㅇㅇ' in g :
#     idx = g.index('ㅇㅇ')
#     del g[idx :]
  
#   else:
#     pass

# sym_spell = SymSpell(max_dictionary_edit_distance = 3)
# dictionary_path = '만개의레시피_재료_목록_오타처리용_decomposed.txt'
# sym_spell.load_dictionary(dictionary_path, 0, 1)

# # gom_typo_fix에 오타 교정된 재료 목록 저장
# gom_typo_fix = []
# for g in gom:
#   g_typo_fix = []
#   for term in g:
#     term = split_syllables(term) 
#     suggestions = sym_spell.lookup(term, Verbosity.ALL, max_edit_distance = 3)
#     for sugg in suggestions :
#       if sugg.distance < 2:
#         typofix = join_jamos(sugg.term)
#         g_typo_fix.append(typofix)
#         break
#       else:
#         pass
#   gom_typo_fix.append(g_typo_fix)
  
  
# gom_typo_fix를 다시 피클로 저장
# with open('gom_typo_fix.pkl','wb') as f:
#     pickle.dump(gom_typo_fix,f)


## 결측치에 곰곰 채우기

In [17]:
#곰곰 상품id 추출
gomgom_productId = list(coupang_items.loc[coupang_items['상품'].str.contains('곰곰')]['상품id'].values)

#곰곰 이미지에서 추출한 구성정보 list
with open('gom_typo_fix.pkl', 'rb') as fr:
    gomgom_data = pickle.load(fr)

#상품id + 구성정보
gomgom_df = pd.DataFrame(zip(gomgom_productId, gomgom_data), columns=['상품id', '구성정보'])

### 곰곰 결측치 채우기
for i in range(len(gomgom_df)):
    idx = data_items[data_items['상품id'] == gomgom_productId[i]].index.values
    gomgom = gomgom_df.loc[i,'구성정보']
    data_items.loc[idx, '구성정보'] = ', '.join(gomgom)
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    372 non-null    object
 1   구성정보    331 non-null    object
dtypes: object(2)
memory usage: 5.9+ KB


## 구성정보 전처리 후 재대입

In [18]:
#구성정보 재료 전처리
def p_c_preprocessing(product_composition):
    product_composition = re.sub(r'\\n', '', product_composition)
    product_composition = re.sub(r'\)', ',', product_composition)
    product_composition = re.sub(r'[^ㄱ-ㅣ가-힣a-zA-Z\,\(\:]', ' ', product_composition)
    product_composition = product_composition.strip()
    
    replace_list = ['으로 구성되어 있습니다','으로 구성되어있습니다','로 구성되어 있습니다','로 구성되어 있어요','로 구성되어있습니다']
    for r in replace_list:
        product_composition = product_composition.replace(r, '')
    
    replace_list = ['과', '와']
    for r in replace_list:
        product_composition = product_composition.replace(r, ',') 
    
    # 추가 작업에 따라 추가 여부 결정
    # if ('야채' in product_composition) or ('채소(' in product_composition) or ('채소 :' in product_composition):
    #     replace_list = ['야채 세트', '혼합 채소', '혼합 해물', '채소(', '채소 :']
    #     for r in replace_list:
    #         product_composition = re.sub('[\(\)\:]', ',', product_composition)
    #         product_composition = product_composition.replace(r, '')
    # else:
    product_composition = re.sub('[\(\)\:]', ',', product_composition)

    return product_composition

In [19]:
#요리재료내용 결측치 삭제
data_items_drop = data_items[data_items.loc[:,'구성정보'].isnull()].index
data_items_drop = data_items.drop(data_items_drop)
data_items_drop.reset_index(drop=True, inplace=True)
data_items_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    331 non-null    object
 1   구성정보    331 non-null    object
dtypes: object(2)
memory usage: 5.3+ KB


>> 소고기 == 쇠고기 이건 어디에..?

In [20]:
#요리재료_전처리(new column) 
data_items_drop['구성정보_전처리'] = np.NaN

for idx in range(len(data_items_drop)):
    ingredient = data_items_drop.loc[idx, '구성정보']
    ingredients = p_c_preprocessing(ingredient)
    ingredients = ingredients.split(',')
    
    ingre_lists = []
    for i in ingredients:
        i = i.strip()
        replace_list = ['칼국수 면', '다진 마늘', '대구 곤이', '명태 곤이']
        if i in replace_list:
            i = i.split(' ')
            i = ''.join(i)

        i = i.split(' ')
        if i[-1] == '등' :  # 등을 그대로 두는 것이 나을지 지금처럼 빼는게 나을지..?
            i.pop(-1)
            i = ' '.join(i)
            ingre_lists.append(i)
        else:
            i = ' '.join(i)
            ingre_lists.append(i)
        
    #null값 제거
    ingre_lists = list(filter(None, ingre_lists))

    #요리재료_전처리(new column)에 추가
    data_items_drop.loc[idx, '구성정보_전처리'] = ', '.join(ingre_lists)

data_items_drop.iloc[0:10, :]

,상품id,구성정보,구성정보_전처리
0,4926044090,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면..."
1,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
2,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기, 만두, 생면, 소스 종, 육수, 표고버섯, 팽이버섯, 채소, 배추, 청경..."
3,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물, 채소, 연두부, 다시팩, 소스"
4,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스"
5,6310053892,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기, 콩나물, 대파, 양파, 깻잎, 소스"
6,1637283332,"순두부, 애호박, 소고기, 팽이버섯, 대파, 소스","순두부, 애호박, 소고기, 팽이버섯, 대파, 소스"
7,188130036,"소고기, 당면, 팽이버섯, 소스, 참기름 등으로 구성되어 있습니다.","소고기, 당면, 팽이버섯, 소스, 참기름"
8,1946869683,"애호박, 양파, 대파, 청양고추, 두부, 팽이버섯","애호박, 양파, 대파, 청양고추, 두부, 팽이버섯"
9,6764028903,"기타 수산물가공품(명태알, 대구곤이, 새우), 채소(무, 대파, 청양고추), 소스로...","기타 수산물가공품, 명태알, 대구곤이, 새우, 채소, 무, 대파, 청양고추, 소스"


# 비교를 위한 리스트 생성

In [21]:
# 만개의 레시피 재료 목록 read
ingre_10000 = []
with open('만개의레시피_재료_목록.txt', 'r') as f :
    lines = f.readlines()
    lines = str(lines).split(',')
    for line in lines:
        ingre_10000.append(line) 
len(ingre_10000)

21806

In [22]:
#쿠팡 재료 목록 생성
ingre_coupang_list = data_items_drop['구성정보_전처리'].dropna()
ingre_coupang_lists = ingre_coupang_list.to_list()

ingre_coupangs = []
for item in ingre_coupang_lists:
    ingre_coupang=[]
    items = item.split(', ')
    for i in items:
        ingre_coupang.append(i)
    ingre_coupangs.append(ingre_coupang)

# #쿠팡 재료 목록 저장
# with open('쿠팡_재료_목록.txt', 'w') as f :
#     for ingre in ingre_coupang:
#         f.write(ingre)

In [23]:
#쿠팡에만 있는 재료만
ingre_not_10000 = []
for items in ingre_coupangs :
    not_10000 = []
    for i in items:
        if not i in ingre_10000 and i != '':
            not_10000.append(i)
    ingre_not_10000.append(not_10000)
print(len(ingre_not_10000))
ingre_not_10000

331


[[],
 [],
 ['소스  종', '표고버섯은 중량 기준으로 담겨있어 수량이 상이할 수 있어요'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['기타 수산물가공품'],
 [],
 [],
 ['바지락맛 액상 소스'],
 ['해산물 팩', '야채팩'],
 ['토자연 우사골육수  팩', '참소스  팩'],
 [],
 [],
 [],
 [],
 [],
 ['디포리 국물용팩'],
 [],
 ['야채 세트', '야채 세트'],
 [],
 ['낙곱새전골 소스', '낙지곱창'],
 [],
 [],
 ['반각 가리비', '야채 세트', '야채 세트'],
 ['혼합 채소'],
 ['잡채 면', '간장 배 양념'],
 ['김치찌개 소스'],
 [],
 ['부대찌개 소스'],
 ['야채팩', '청국장  조리 시 물    mL는 추가로 준비해주세요'],
 [],
 ['야채 세트', '바른 마늘'],
 [],
 ['김치 왕만두', '혼합 채소', '혼합 해물', '자숙홍합살'],
 [],
 ['정통 순두부찌개순두부', '순두부 찌개양념장'],
 ['야채 세트', '야채 세트'],
 ['소고기 포장육', '소스  종'],
 [],
 [],
 ['주름 미더덕', '얼큰 분말 스프'],
 [],
 ['야채세트', '바른마늘'],
 ['야채 세트'],
 [],
 ['닭고기 포장육', '수산물가공품'],
 ['야채 세트', '바른 마늘'],
 ['야채세트', '바른마늘'],
 ['백목이버섯'],
 [],
 ['분말 수프'],
 ['야채 세트', '야채 세트'],
 [],
 ['마늘 양념 소곱창'],
 ['고기국수육수  팩', '새우맛양념  팩'],
 ['바지락 맛 액상 소스'],
 [],
 ['야채 세트', '건조 매생이'],
 [],
 [],
 ['야채 세트'],
 ['알탕 전용 소스'],
 ['얼큰육수베이스  팩', '새우맛양념  팩'],
 ['야채세트'],
 ['야채 세트'],
 ['야채 세트', '바른 마늘'],
 ['찌개 두부'],
 ['매운탕 양념'],
 ['기타 

In [24]:
data_items_drop['구성정보_전처리']

0      쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...
1      양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...
2      소고기, 만두, 생면, 소스  종, 육수, 표고버섯, 팽이버섯, 채소, 배추, 청경...
3                                   해물, 채소, 연두부, 다시팩, 소스
4             소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스
                             ...                        
326    고기 손만두, 소고기, 채소, 청경채, 대파, 청양고추, 홍고추, 버섯, 팽이버섯,...
327    소고기, 스키야키 육수, 해물 유부주머니, 두부 스틱, 폰즈 소스, 우동사리, 야채...
328    새우, 야채 세트, 양파, 마늘, 건고추, 하얀 설탕, 올리브 오일, 버터 후레시,...
329     쇠고기, 느타리버섯, 미나리, 팽이버섯, 감자, 양파, 샤브용얼큰소스, 샤브샤브재료육수
330    새우, 홍합, 비엔나소시지, 아침 찰옥수수, 야채 세트, 감자, 양파, 마늘, 하얀...
Name: 구성정보_전처리, Length: 331, dtype: object

In [25]:
#일치하는 재료 리스트로 만들고 데이터 프레임으로 수정
ingre_10000s = []
for items in ingre_coupangs :
    ingre_10000_list = []
    for i in items:
        if i in ingre_10000:
            ingre_10000_list.append(i)
    ingre_10000s.append(ingre_10000_list)
print(len(ingre_10000s))
ingre_10000s

331


[['쇠고기',
  '새송이버섯',
  '표고버섯',
  '초간장소스',
  '칠리소스',
  '샤브용육수',
  '생면',
  '칼국수면',
  '숙면',
  '감자수제비',
  '깻잎',
  '팽이버섯',
  '배추',
  '청경채'],
 ['양념육',
  '소곱창',
  '소대창',
  '전골용 육수',
  '새송이버섯',
  '느타리버섯',
  '소고기',
  '두부',
  '배추',
  '양파',
  '깻잎',
  '라면사리'],
 ['소고기', '만두', '생면', '육수', '표고버섯', '팽이버섯', '채소', '배추', '청경채', '깻잎'],
 ['해물', '채소', '연두부', '다시팩', '소스'],
 ['소시지', '오뗄팜S', '프레스햄', '소고기', '치즈', '라면사리', '채소', '김치', '소스'],
 ['소고기', '콩나물', '대파', '양파', '깻잎', '소스'],
 ['순두부', '애호박', '소고기', '팽이버섯', '대파', '소스'],
 ['소고기', '당면', '팽이버섯', '소스', '참기름'],
 ['애호박', '양파', '대파', '청양고추', '두부', '팽이버섯'],
 ['명태알', '대구곤이', '새우', '채소', '무', '대파', '청양고추', '소스'],
 ['소고기', '소스', '쥬키니호박', '팽이버섯', '대파', '청양고추', '두부'],
 ['두부', '청국장', '애호박', '돼지고기', '대파', '양파', '혼합장'],
 ['바지락', '생합', '모시조개', '파', '고추', '무'],
 ['누룽지', '단호박', '청경채', '콩기름', '소스'],
 ['소 스지', '전각 슬라이스'],
 ['돼지고기', '모듬햄', '대파', '양파', '양배추', '라면사리', '부대찌개소스', '김치', '슬라이스', '치즈'],
 ['미역', '쇠고기', '육수', '볶음', '맛기름'],
 ['두부', '쇠고기', '소스', '팽이버섯', '청양고추', '애호박', '대파', 

In [26]:
for idx in range(len(ingre_10000s)):
    data_items_drop.loc[idx, '구성정보_전처리'] = ','.join(ingre_10000s[idx])
data_items_drop.head()

,상품id,구성정보,구성정보_전처리
0,4926044090,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수..."
1,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
2,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎"
3,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"
4,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"


## 긴 텍스트에서 사전과 일치하는 요리재료만 추출

In [27]:
#요리재료 사전과 일치여부를 통해 요리재료 추출
# ingredients_list = []
# for coupangs in ingre_coupangs :
#     ingre_list = []
#     for coupang in coupangs:
#         if coupang in ingre_10000:
#             ingre_list.append(coupang)
#         else:
#             ingre = [ man for man in ingre_10000 if man in coupang]
#             ingre_list.extend(ingre)
#     ingredients_list.append(ingre_list)
#     print(ingre_list)
#ingredients_list

# 쿠팡 상품 리스트와 구성정보 병합

In [28]:
#쿠팡 상품 리스트와 구성정보 병합
coupang = pd.merge(coupang_items, data_items_drop)
coupang.to_csv('coupang_filtering.csv')
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 0 to 330
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                331 non-null    object
 1   상품id                 331 non-null    object
 2   data-item-id         331 non-null    object
 3   data-vendor-item-id  331 non-null    object
 4   상품                   331 non-null    object
 5   상품명                  331 non-null    object
 6   정가                   331 non-null    object
 7   할인율                  331 non-null    object
 8   판매가                  331 non-null    object
 9   100g당_가격             290 non-null    object
 10  별점                   331 non-null    object
 11  리뷰수                  331 non-null    object
 12  품절여부                 331 non-null    object
 13  구성정보                 331 non-null    object
 14  구성정보_전처리             331 non-null    object
dtypes: object(15)
memory usage: 41.4+ KB


In [29]:
coupang.head()

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
0,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15752,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수..."
1,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11339,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
2,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,17%,13900,"(100g당 1,231원)",4.5,49488,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎"
3,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12127,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"
4,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,36633,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"


In [30]:
#구성정보 결측치 확인
print('결측치 개수: ',len(coupang.loc[coupang['구성정보_전처리']=='']))
coupang.loc[coupang['구성정보_전처리']=='']

결측치 개수:  9


,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
83,1,2140731697,3635578248,71621124717,설래담 민어 매운탕 (냉동),"설래담 민어 매운탕 (냉동), 680g, 1개",15190,0,15190,"(100g당 2,234원)",4.0,336,0,"신선한 민어, 풍성한 야채 믹스, 얼큰한 풍미의 매운탕 소스, 시원한 맛을 더해줄 ...",
143,3,5965059575,10701364629,77982083715,마이셰프 명절 밀키트 버섯가득 소불고기전골 905g + 소고기버섯잡채 571g + ...,마이셰프 명절 밀키트 버섯가득 소불고기전골 905g + 소고기버섯잡채 571g + ...,43700,15%,36900,NaN,4.5,21,0,"버섯가득 소불고기 전골, 궁중 소불고기, 소고기 버섯 잡채로 구성되어 있습니다.",
144,3,5965059546,10701364550,77982083557,마이셰프 명절 밀키트 궁중 소고기 갈비찜 1026g + 소고기버섯잡채 571g + ...,마이셰프 명절 밀키트 궁중 소고기 갈비찜 1026g + 소고기버섯잡채 571g + ...,66700,11%,58900,NaN,5.0,24,0,"궁중 소고기 갈비찜, 일품 소고기 육전, 소고기 버섯 잡채로 구성되어 있습니다.",
250,7,6703653767,15536769131,82755955008,프렙 이탈리안 감바스 알 아히요 (냉동),"프렙 이탈리안 감바스 알 아히요 (냉동), 300g, 1개",13200,0,13200,"(100g당 4,400원)",4.5,48,0,포장 정보,
265,7,6928382170,16756449794,83937960368,셰프초이스 어제 버무린 매콤한 오징어볶음 씨키트 750g,"셰프초이스 어제 버무린 매콤한 오징어볶음 씨키트 750g, 1세트",14900,0,14900,NaN,1.0,2,0,"매콤한 오징어볶음, 자른 대파, 자른 양배추, 사리 소면으로 구성되어 있습니다.",
266,7,6928382821,16756453661,83937964890,셰프초이스 어제 버무린 오리지널 오징어볶음 씨키트 750g,"셰프초이스 어제 버무린 오리지널 오징어볶음 씨키트 750g, 1세트",14900,0,14900,NaN,5.0,1,0,"오리지널 오징어볶음, 자른 대파, 자른 양배추, 사리 소면으로 구성되어 있습니다.",
270,7,6703653639,15536768551,82755954355,프렙 버터 전복 리조또 (냉동),"프렙 버터 전복 리조또 (냉동), 427g, 1개",11600,17%,9600,"(100g당 2,248원)",4.5,60,0,포장 정보,
276,7,6703653501,15536767789,82755953691,프렙 볼로네제 리조또 (냉동),"프렙 볼로네제 리조또 (냉동), 520g, 1개",11200,0,11200,"(100g당 2,154원)",4.5,42,0,포장 정보,
314,7,6811645649,16116095590,83313001742,집반찬연구소 간편 짜장용 손질야채 240g x 2개 + 청정원 직화 짜장 80g 세트,집반찬연구소 간편 짜장용 손질야채 240g x 2개 + 청정원 직화 짜장 80g 세...,7900,0,7900,NaN,5.0,6,1,청정원 직화짜장과 손질 야채로 구성되어 있습니다.,


In [31]:
#우선 결측치 제거
coupang_null_list = coupang.loc[coupang['구성정보_전처리']==''].index
coupang.drop(coupang_null_list, inplace=True)



In [32]:
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 0 to 330
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                322 non-null    object
 1   상품id                 322 non-null    object
 2   data-item-id         322 non-null    object
 3   data-vendor-item-id  322 non-null    object
 4   상품                   322 non-null    object
 5   상품명                  322 non-null    object
 6   정가                   322 non-null    object
 7   할인율                  322 non-null    object
 8   판매가                  322 non-null    object
 9   100g당_가격             286 non-null    object
 10  별점                   322 non-null    object
 11  리뷰수                  322 non-null    object
 12  품절여부                 322 non-null    object
 13  구성정보                 322 non-null    object
 14  구성정보_전처리             322 non-null    object
dtypes: object(15)
memory usage: 40.2+ KB


# 상세카테고리 추가

In [33]:
# def category_name(product):
    
#     if '전골' in product : name = '전골'
#     elif '나베' in product : name = '전골'
#     elif '해물' in product : name = '해물'
#     elif '마라' in product : name = '마라'
#     elif '탕' in product : name = '탕'
#     elif '찌개' in product : name = '찌개'
#     elif '국수' in product : name = '면'
#     elif '국' in product : name = '국'
#     elif '육개장' in product : name = '국'
#     elif '파개장' in product : name = '국'
#     elif '파스타' in product : name = '파스타'
#     elif '짜글이' in product : name = '기타'

#     elif '면' in product : name = '면'
#     elif '소바' in product : name = '소바'
#     elif '떡볶이' in product : name = '떡볶이'
    
#     elif '보쌈' in product : name = '보쌈'
#     elif '족발' in product : name = '족발'
#     elif '순대' in product : name = '순대'
#     elif '마파두부' in product : name = '마파두부'
#     elif '양장피' in product : name = '양장피'
#     elif '씨푸드' in product : name = '씨푸드'
#     elif '씨키트' in product : name = '씨푸드'
#     elif '콩불' in product : name = '콩불'
#     elif '콩나물 불고기' in product : name = '콩불'
#     elif '라자냐' in product : name = '라자냐'
    
#     elif '리조또' in product : name = '리조또'
#     elif '닭갈비' in product : name = '닭갈비'
#     elif '조림' in product : name = '조림'
#     elif '오리' in product : name = '오리'
#     elif '라면' in product : name = '라면'
#     elif '불고기' in product : name = '불고기'
#     elif '샤브샤브' in product : name = '샤브샤브'
#     elif '새우' in product : name = '씨푸드'
#     elif '커리' in product : name = '카레'
#     elif '카레' in product : name = '카레'
#     elif '수제비' in product : name = '수제비'
#     elif '찜' in product : name = '찜'
#     elif '비빔밥' in product : name = '비빔밥'
#     elif '부침개' in product : name = '부침개'
#     elif '샌드위치' in product : name = '샌드위치'
#     elif '쌈' in product : name = '쌈'
#     elif '무침' in product : name = '무침'
#     elif '스키야키' in product : name = '스키야키'
#     elif '연어' in product : name = '씨푸드'
#     elif '쉬림프' in product : name = '씨푸드'
#     elif '누들' in product : name = '면'
#     elif '스테이크' in product : name = '스테이크'
#     elif '짜장' in product : name = '면'
#     elif '짬뽕' in product : name = '면'
#     elif '백숙' in product : name = '백숙'
#     elif '감바스' in product : name = '감바스'
#     elif '볶음' in product : name = '볶음'
#     elif '조개' in product : name = '씨푸드'
#     elif '문어' in product : name = '씨푸드'
#     elif '양장피' in product : name = '양장피'
#     elif '우동' in product : name = '면'
#     elif '잡채' in product : name = '잡채'
#     elif '떡볶잉' in product : name = '떡볶이'
#     elif '구이' in product : name = '구이'
#     elif '닭' in product : name = '닭'
#     elif '덮밥' in product : name = '덮밥'
#     elif '꽃게' in product : name = '씨푸드'
#     elif '분식' in product : name = '분식'
#     elif '오코노미야끼' in product : name = '오코노미야끼'
#     else: name='기타'
        
#     return name

In [89]:
def category_name(product):
    if ('닭' in product) or ('백숙' in product) : name = '닭'
    elif '구이' in product : name = '구이'
    elif ('전골' in product)or('나베' in product)or('스키야키' in product) or ('샤브샤브' in product): name = '전골'
    elif '족발' in product : name = '족발'
    elif '국수' in product : name = '국수'
    elif ('해물탕' in product) or ('연포탕' in product) or ('해신탕' in product) or ('꽃게탕' in product) or ('해물 누룽지탕' in product) or ('조개탕' in product) or ('바지락탕' in product): name = '해물탕'
    elif ('해물뚝배기' in product) or ('문어오리탕' in product) or ('홍합탕' in product) : name = '해물탕'
    elif ('매운탕' in product) or ('대구탕' in product) or ('동태탕' in product) or ('지리탕' in product)  or ('맑은탕' in product) or ('맑음탕' in product): name = '생선탕'
    elif ('알탕' in product) or ('알이 맛있탕' in product) or ('곤이탕' in product): name = '알탕'

    elif ('찌개' in product) or ('짜글이' in product) or ('짬뽕 순두부' in product): name = '찌개/짜글이'

    elif ('면' in product)or('소바' in product)or('짜장' in product)or('누들' in product)or('짬뽕' in product) : name = '면'
    elif ('우동' in product)or('라면' in product) : name = '면'
    elif '스테이크' in product : name = '스테이크'

    elif ('파스타' in product) or ('라자냐' in product) : name = '파스타'
    


    elif '리조또' in product : name = '리조또' 
        
    elif '마라탕' in product : name = '마라탕'
    elif '마파두부' in product : name = '마파두부'

    elif ('떡볶이' in product) or ('떡볶잉' in product) : name = '떡볶이'
        
    elif '덮밥' in product : name = '덮밥'
    elif '순대' in product : name = '순대'
        
    elif '무침' in product : name = '무침'
    elif '감바스' in product : name = '감바스'
    elif ('해물' in product)or('씨푸드' in product)or('씨키트' in product)or('꽃게' in product)or('새우' in product): name = '씨푸드'
    elif ('쉬림프' in product)or('연어' in product)or('조개' in product)or('문어' in product): name = '씨푸드'

    elif ('콩불' in product) or ('불고기' in product) or ('비빔밥' in product) : name='한식'
    elif ('부침개' in product) or ('조림' in product) : name = '한식'

    elif '오리' in product : name = '오리'

    elif ('커리' in product) or ('카레' in product) : name = '카레'
    
    elif '샌드위치' in product : name = '샌드위치'
    elif ('볶음' in product) or ('잡채' in product) : name = '볶음'
    
    elif '쌈' in product : name = '쌈'
    elif ('국' in product) or ('육개장' in product) or ('파개장' in product) or ('수제비' in product) or ('칼제비' in product) : name = '국'
    elif '찜' in product : name = '찜'
    else: name='기타'
        
    return name

In [90]:
coupang['상세카테고리'] = coupang['상품'].apply(lambda x : category_name(x))
coupang.loc[coupang['상세카테고리'] == '기타']

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
14,1,6617266915,15030233120,82252852910,복선당 규스지 사골 도가니탕 2인분 1160g + 소스 20g x 2p 세트 (냉동),복선당 규스지 사골 도가니탕 2인분 1160g + 소스 20g x 2p 세트 (냉동...,20250,9%,18280,NaN,4.5,183,0,"소 스지와 전각 슬라이스, 토자연 우사골육수 2팩, 참소스 2팩으로 구성되어 있습니다.","소 스지,전각 슬라이스",기타
42,1,6641443811,15183074783,82404326684,마이셰프 TREASURE HUNTER 우정분식 물떡 어묵탕 2인분 (냉동),"마이셰프 TREASURE HUNTER 우정분식 물떡 어묵탕 2인분 (냉동), 372...",14400,10%,12900,"(100g당 3,468원)",4.5,73,0,"어묵, 떡, 대파, 무, 건고추, 소스로 구성되어 있습니다.","어묵,떡,대파,무,건고추,소스",기타
53,1,6869596963,16429102275,83619914910,홈스토랑 부산식 물떡 & 꼬치 어묵탕 (냉동),"홈스토랑 부산식 물떡 & 꼬치 어묵탕 (냉동), 876g, 1개",13840,0,13840,"(100g당 1,580원)",5.0,14,0,"어묵, 떡, 중화면, 무, 대파, 다시마, 건고추, 소스로 구성되어 있습니다.","어묵,떡,중화면,무,대파,다시마,건고추,소스",기타
74,1,6645647324,15208476612,82429535407,오마뎅 그시절 오뎅탕 (냉동),"오마뎅 그시절 오뎅탕 (냉동), 1개, 550g",13900,0,13900,"(100g당 2,527원)",5.0,203,0,다채로운 모양의 어묵들과 소스로 구성되어 있습니다.,소스,기타
210,4,6135492876,11729969996,79003935874,애슐리 에그인헬,"애슐리 에그인헬, 873g, 1개",14280,30%,9990,"(100g당 1,144원)",4.5,314,1,"수란, 건면, 빵, 채소(양파, 대파), 소시지, 건고추, 베이컨, 마늘, 파슬리,...","수란,건면,빵,채소,양파,대파,소시지,건고추,베이컨,마늘,파슬리,올리브유,치즈,소스",기타
219,6,293474370,926574632,5302429594,마이셰프 중화풍 양장피,"마이셰프 중화풍 양장피, 600g, 1개",16900,0,16900,"(100g당 2,817원)",4.5,11357,1,"1. 표고버섯, 새우, 오징어, 양장피 면, 건목이버섯, 참기름, 고기, 양파, ...","표고버섯,새우,오징어,양장피 면,건목이버섯,참기름,고기,양파",기타
251,7,6538908877,14550365328,81792721798,푸드어셈블 채선당 분짜 맛집키트 2인분,"푸드어셈블 채선당 분짜 맛집키트 2인분, 585g, 1개",15900,3%,15400,"(100g당 2,632원)",4.5,76,0,"소고기, 야채 3종, 적양배추, 깻잎, 건면, 소스 2종, 절임식품으로 구성되어 있...","소고기,적양배추,깻잎,건면,빨강파프리카,청피망,노랑파프리카",기타
263,7,6740798714,15735464977,82949395099,푸드어셈블 맛집키트 쏭타이 팟싸완 비프팟타이 밀키트,"푸드어셈블 맛집키트 쏭타이 팟싸완 비프팟타이 밀키트, 752g, 1개",15400,1%,15100,"(100g당 2,008원)",5.0,21,0,"건면, 숙주나물, 고수, 소고기, 단무지, 건새우, 계란, 땅콩분태, 크러쉬드레드페...","건면,숙주나물,고수,소고기,단무지,건새우,계란,땅콩분태,크러쉬드레드페퍼,레몬주스,대...",기타
284,7,6641444058,15183076854,82404328814,마이셰프 TREASURE HUNTER 우정분식 꿀꺽 야끼만두 2인분 (냉동),"마이셰프 TREASURE HUNTER 우정분식 꿀꺽 야끼만두 2인분 (냉동), 1팩...",8900,33%,5900,"(100g당 2,458원)",3.5,9,0,"잡채 튀김, 소스로 구성되어 있습니다.",소스,기타
289,7,6410059410,13743233827,80994055849,쉐푸드 유자소스 멕시칸 파히타 밀키트 (냉동),"쉐푸드 유자소스 멕시칸 파히타 밀키트 (냉동), 655g, 1개",17900,0,17900,"(100g당 2,733원)",4.5,10,1,"또띠아, 소스, 올리브오일, 흰다리새우살, 닭가슴살, 채소(양파, 황파프리카, 홍파...","또띠아,소스,올리브오일,흰다리새우살,닭가슴살,채소,양파,황파프리카,홍파프리카,청피망",기타


In [62]:
coupang['상세카테고리']

array(['전골', '생선탕', '한식', '찌개/짜글이', '알탕', '해물탕', '기타', '국', '구이', '순대',
       '씨푸드', '닭', '찜', '오리', '면', '국수', '스테이크', '파스타', '감바스', '떡볶이',
       '마라탕', '마파두부', '무침', '쌈', '샌드위치', '볶음', '족발', '리조또', '카레'],
      dtype=object)

In [ ]:
dict1 = {'찌개/국' : '전골, 생선탕, 찌개/짜글이, 알탕, 해물탕, 국, 마라탕',
         '면/파스타' : '면, 국수, 파스타',
         '구이' : '구이, 스테이크',
         '볶음/튀김' : '볶음, 감바스, 순대',
         '조림/찜' : '찜, 마파두부, 족발',
         '전체' : '한식, 기타, 씨푸드, 닭, 오리, 떡볶이, 무침, 쌈, 샌드위치, 리조또, 카레'}



In [37]:
dddd = pd.read_csv('2022-12-23navertrend.csv')

In [38]:
dddd

,Unnamed: 0,전체,찌개/국,면/파스타,구이,볶음/튀김,조림/찜
0,0,밀키트,밀키트,밀키트,밀키트,밀키트,밀키트
1,1,떡볶이,낙곱새,수제비밀키트,스테이크밀키트,감바스밀키트,보일링크랩
2,2,낙곱새,알탕,동남아식밀키트탄탄면,자미곱,닭갈비밀키트,찜닭밀키트
3,3,알탕,캠핑밀키트,공항칼국수,군고기밀키트,애슐리밀키트,갈비찜오리지널맛
4,4,쌀떡볶이,해신탕,파스타밀키트,목살밀키트,감바스,캠핑요리추천
5,5,캠핑밀키트,충무로한우곱창전골,소고기버섯칼국수,석쇠직화술안주캠핑음식술안주,돈이곱창참숯초벌냄새없는곱창볶음순대볶음간단캠핑요리밀키트돼지곱창볶음600g,캠핑음식추천
6,6,해신탕,동태밥상,애슐리밀키트,초벌한입닭갈비,송탄춘천닭갈비밀키트,우족찜
7,7,국물떡볶이,만두전골,라자냐밀키트,애슐리밀키트,닭갈비,소갈비찜밀키트
8,8,스테이크밀키트,곱창전골,라자냐,자미곱대창,춘천닭갈비택배생생닭갈비3인분900g캠핑요리,바지락술찜
9,9,충무로한우곱창전골,더원푸드시래기곱창전골밀키트,뇨끼밀키트,프레시지,프레시지밀키트,아구찜


1. 띄어쓰기 텍스트 정리
2. 쿠팡 상세카테고리 - 네이버트렌드 카테고리 매칭
3. 추천시스템 모델

ex) 세부카테고리 전골카테고리안의 쿠팡 밀키트와, - 트렌드의 찌개/국칼럼의 모든 트렌드데이터 비교 
상품 설명을 가져온다?b

In [39]:
coupang

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
0,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15752,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골
1,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11339,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골
2,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,17%,13900,"(100g당 1,231원)",4.5,49488,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎",전골
3,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12127,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스",생선탕
4,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,36633,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스",전골
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,7,6410059385,13743233647,80994055823,쉐푸드 배추 만두 전골 밀키트 (냉동),"쉐푸드 배추 만두 전골 밀키트 (냉동), 968g, 1개",18720,0,18720,"(100g당 1,934원)",4.5,74,1,"고기 손만두, 소고기, 채소(청경채, 대파, 청양고추, 홍고추), 버섯(팽이버섯, ...","소고기,채소,청경채,대파,청양고추,홍고추,버섯,팽이버섯,새송이버섯,느타리버섯,알배추...",전골
327,7,6407831462,13729238438,80980206078,마이셰프 알찬 소고기 스키야키,"마이셰프 알찬 소고기 스키야키, 1070g, 1개",16900,5%,15900,"(100g당 1,486원)",4.5,215,1,"소고기, 스키야키 육수, 해물 유부주머니와 두부 스틱, 폰즈 소스, 우동사리, 야채...","소고기,우동사리,야채,청경채,배추,대파,숙주나물,팽이버섯",전골
328,7,6842831802,16276794016,83469464083,바다자리 생새우 갈릭버터쉬림프 1~2인분,"바다자리 생새우 갈릭버터쉬림프 1~2인분, 300g, 1개",21600,0,21600,"(100g당 7,200원)",5.0,3,1,"새우, 야채 세트(양파, 마늘), 건고추, 하얀 설탕, 올리브 오일, 버터 후레시,...","새우,양파,마늘,건고추,올리브 오일,허브 솔트",씨푸드
329,7,6961893890,16951928882,84129277131,바른식 등촌식 얼큰 미나리 샤브샤브 (냉장),"바른식 등촌식 얼큰 미나리 샤브샤브 (냉장), 1개, 800g",16800,0,16800,"(100g당 2,100원)",5.0,2,1,"쇠고기, 느타리버섯, 미나리, 팽이버섯, 감자, 양파, 샤브용얼큰소스, 샤브샤브재료...","쇠고기,느타리버섯,미나리,팽이버섯,감자,양파",전골


In [40]:
docs = coupang.상품.values

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

In [42]:
countvect = vect.fit_transform(docs)
countvect

<322x586 sparse matrix of type '<class 'numpy.int64'>'
	with 1543 stored elements in Compressed Sparse Row format>

In [43]:
sorted(vect.vocabulary_)

['02',
 '1160g',
 '150g',
 '15g',
 '1kg',
 '1인분',
 '200g',
 '20g',
 '240g',
 '250g',
 '2in1',
 '2p',
 '2인',
 '2인분',
 '300g',
 '320g',
 '340g',
 '3인분',
 '450g',
 '4인분',
 '4종',
 '500g',
 '540g',
 '545g',
 '550g',
 '560g',
 '588g',
 '5분키친',
 '600g',
 '602g',
 '620g',
 '650g',
 '680g',
 '690g',
 '6컵',
 '700g',
 '750g',
 '765g',
 '800g',
 '850g',
 '8분완성',
 '900g',
 '950g',
 '960g',
 '99',
 '99프레시밀키트',
 'apeach',
 'ebs',
 'fresh',
 'hunter',
 'mychef',
 'the',
 'theking',
 'the청국장찌개',
 'treasure',
 'ufo',
 '가니시',
 '가득',
 '간편',
 '간편한',
 '간편한식',
 '갈릭버터쉬림프',
 '갈릭비프스테이크',
 '갈치구이',
 '갈치조림',
 '감바스',
 '감바스알아히오',
 '감바스알아히요',
 '감자',
 '감자수제비',
 '갑오징어파스타',
 '갓포아키',
 '게살',
 '겨울엔',
 '고구마',
 '고기',
 '고기듬뿍',
 '고깃집',
 '고등어구이',
 '고잉메리',
 '고추',
 '고추잡채',
 '고추잡채와',
 '고추장',
 '고추장찌개',
 '곤드레',
 '골든비프',
 '곰곰',
 '곰표',
 '곱창',
 '곱창전골',
 '관자',
 '관자파스타',
 '광주식',
 '국내산',
 '국물',
 '국물떡볶이',
 '국산갈치구이',
 '국산깐새우',
 '국산새우',
 '국수',
 '굴부추전',
 '굴비매운탕',
 '굴전골',
 '굴짬뽕탕',
 '규스지',
 '그시절',
 '기사식당',
 '기승전골',
 '김치',
 '김치부침개',
 '김치어묵',
 '김

In [44]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

countvect_df

,02,1160g,150g,15g,1kg,1인분,200g,20g,240g,250g,...,햄가득,햄치즈,허챠밍,호로록,호밍스,홈스토랑,홈파티,홍합탕,활가리비술찜,활바지락술찜
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
319,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# 위의 Data Frame 형태의 유사도를 계산 

from sklearn.metrics.pairwise import cosine_similarity

cosine_matrix = cosine_similarity(countvect_df, countvect_df)

In [46]:
productid = {}
for i, c in enumerate(coupang['상품']): productid[i] = c

product = {}
for i, c in productid.items(): product[c] = i

In [47]:
product

{'곰곰 밀푀유 나베': 0,
 '프레시지 대한곱창 곱창전골': 1,
 '마이셰프 밀푀유나베 & 칼국수': 2,
 '미소프레쉬 6컵 푸짐한 대구탕': 3,
 '프레시지 더큰 햄가득 부대전골': 4,
 '마이셰프 EBS 최요비 산더미 소고기 콩불 2인분': 5,
 '곰곰 우삼겹 순두부 찌개': 6,
 '프레시지 서울식 불고기 전골': 7,
 '곰곰 애호박 된장찌개 밀키트': 8,
 '프레시지 톡톡 터지는 알탕 (냉동)': 9,
 '프레시지 고깃집 된장찌개': 10,
 '곰곰 간편한 청국장찌개 밀키트': 11,
 '바다자리 모둠조개탕 밀키트 2~3인분 550g': 12,
 '앙트레 해물 누룽지탕 (2인분)': 13,
 '복선당 규스지 사골 도가니탕 2인분 1160g + 소스 20g x 2p 세트 (냉동)': 14,
 '곰곰 옛날식 부대찌개': 15,
 '곰곰 소고기 된장찌개 밀키트': 16,
 '곰곰 한우 미역국 밀키트': 17,
 '곰곰 깊은맛 곱창전골 밀키트': 18,
 '프레시밀 샤브샤브재료': 19,
 '설래담 낙지 연포탕용 (냉동)': 20,
 '삼초마을 고기듬뿍 고추장찌개 (냉동)': 21,
 '바다자리 동태알탕용 씨키트 3~4인분': 22,
 '프레시지 우삼겹 순두부찌개': 23,
 '청정원 호밍스 낙곱새전골 (냉동)': 24,
 '푸드어셈블 채선당 불고기 꽃만두 전골 & 칼국수 (냉동)': 25,
 '프레시지 자이언트 부대찌개': 26,
 '바다자리 해물가득 문어해물탕용 씨키트 3~4인분': 27,
 '쿡킷 얼큰 버섯 비비고 만두전골 2인분 (냉동)': 28,
 '쿡킷 육수가득 소불고기 전골 2인분 (냉동)': 29,
 '푸드어셈블 더 알찬 연남동 통삼겹 김치찌개 밀키트': 30,
 '곰곰 샤브 버섯 국수 전골 (냉장)': 31,
 '앙트레 우삼겹 부대찌개 2인분': 32,
 '앙트레 쿠킹박스 간편한식 청국장찌개': 33,
 '풀무원 얼큰 순두부찌개 킷': 34,
 '바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채': 35,
 

In [48]:
idx = product['프레시지 대한곱창 곱창전골'] 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:20] # 상위 10개의 인덱스와 유사도를 추출 

[(222, 0.408248290463863),
 (232, 0.408248290463863),
 (10, 0.3333333333333334),
 (23, 0.3333333333333334),
 (26, 0.3333333333333334),
 (43, 0.3333333333333334),
 (237, 0.3333333333333334),
 (271, 0.3333333333333334),
 (4, 0.2886751345948129),
 (7, 0.2886751345948129),
 (18, 0.2886751345948129),
 (49, 0.2886751345948129),
 (52, 0.2886751345948129),
 (60, 0.2886751345948129),
 (72, 0.2886751345948129),
 (78, 0.2886751345948129),
 (125, 0.2886751345948129),
 (153, 0.2886751345948129),
 (166, 0.2886751345948129),
 (219, 0.2886751345948129)]

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

docs = coupang.구성정보_전처리.values
tfid = TfidfVectorizer()
tfvect = tfid.fit_transform(docs)
tfvect_df = pd.DataFrame(tfvect.toarray(), columns = sorted(tfid.vocabulary_))


tfvect_df

,가리비살,간마늘,간장소스,갈치,감귤,감자,감자수제비,갑오징어,건고추,건면,...,홍합살,화유,황파프리카,후추가루,훈제,훈제오리,흑목이버섯,흑임자,흰다리새우,흰다리새우살
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.335854,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.301846,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.427585,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
cosine_matrix = cosine_similarity(tfvect_df, tfvect_df)

productid = {}
for i, c in enumerate(coupang['구성정보_전처리']): productid[i] = c

product = {}
for i, c in productid.items(): product[c] = i


idx = product['쇠고기,대파,팽이버섯'] 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:20] # 상위 10개의 인덱스와 유사도를 추출 

[(17, 0.5949283811008335),
 (320, 0.5553648730223011),
 (31, 0.42022207476708023),
 (182, 0.352231735600279),
 (287, 0.3221169915669162),
 (8, 0.3181717055359357),
 (6, 0.3147536027617376),
 (82, 0.3073473921896893),
 (10, 0.2962555319685894),
 (0, 0.29443838736280753),
 (181, 0.2903801407835445),
 (18, 0.28939281656134896),
 (23, 0.28762840483894947),
 (112, 0.28456123071832595),
 (16, 0.27427572407458284),
 (138, 0.247942893693692),
 (115, 0.24223221352691587),
 (201, 0.23484798927912576),
 (89, 0.23101472164196507),
 (7, 0.22624666605267577)]

In [51]:
docs = coupang.구성정보_전처리.values
cv = CountVectorizer()
cvtr = cv.fit_transform(docs)
cvtr_df = pd.DataFrame(cvtr.toarray(), columns = sorted(cv.vocabulary_))

In [52]:
cosine_matrix = cosine_similarity(cvtr_df, cvtr_df)

productid = {}
for i, c in enumerate(coupang['구성정보_전처리']): productid[i] = c

product = {}
for i, c in productid.items(): product[c] = i


idx = product['쇠고기,대파,팽이버섯'] 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:20] # 상위 10개의 인덱스와 유사도를 추출 

[(17, 0.6123724356957946),
 (287, 0.596284793999944),
 (31, 0.5477225575051662),
 (6, 0.4714045207910318),
 (8, 0.4714045207910318),
 (320, 0.4714045207910318),
 (10, 0.4364357804719848),
 (23, 0.4364357804719848),
 (82, 0.4364357804719848),
 (104, 0.4364357804719848),
 (112, 0.4364357804719848),
 (115, 0.408248290463863),
 (201, 0.408248290463863),
 (298, 0.408248290463863),
 (318, 0.408248290463863),
 (89, 0.3849001794597505),
 (103, 0.3849001794597505),
 (128, 0.3849001794597505),
 (182, 0.3849001794597505),
 (48, 0.3651483716701108)]

In [53]:

from soynlp.utils import DoublespaceLineCorpus

corpus_fname = '2016-10-20.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)


223357

1. 네이버 트렌드 새로 데이터 추출해서, 띄어쓰기 사전 보충

2. 네이버 트렌드 데이터로 추천시스템 돌려보기

3. 조건식을 통해 유사도가 특정 수치 이하면, 그냥 패스시켜주고 그 카테고리인 애들 보여주는 정도? 여러 개중 리뷰수 기준?

